In [1]:
# Bibliotecas de uso general en el cuaderno
import json
import pickle
import os
import osmnx as ox
import geopandas as gpd
from myutils import *
from numpy import random

random.seed(0)

In [2]:
# Directorio de datos generales
path = "../GeoData/"

# Carga el grafo de la red de transporte de la Ciudad de México
g = ox.load_graphml(filepath = path + "graph_transport.graphml")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(g, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [3]:
# Carga las pltbs
pltbs = gpd.read_file(path + 'PLTBs.gpkg', layer = 'PLTBs_nodes')

# Serializa los datos de tipo lista
pltbs['streets'] = pltbs['streets'].apply(json.loads)
pltbs['oneway'] = pltbs['oneway'].apply(json.loads)
pltbs['length'] = pltbs['length'].apply(json.loads)
pltbs['capacity'] = pltbs['capacity'].apply(json.loads)
pltbs['grouped'] = pltbs['grouped'].apply(json.loads)
pltbs_grupos= [item for sublist in [ i for i in pltbs['grouped']] for item in sublist]
pltbs_nodos = list(pltbs['node'])
pltbs.set_index('node', inplace=True)
print('Grafo de transporte y PLTBs cargados')

Grafo de transporte y PLTBs cargados


In [4]:
voronoi_pltbs = nx.voronoi_cells(g, pltbs_nodos, weight='weight')
unreachable = voronoi_pltbs["unreachable"]
del(voronoi_pltbs)
g = g.remove_nodes_from(unreachable)

In [6]:
traffic_path = '../GeoData/Traffic//TrafficDataAuto/'
traffic_files = [file for file in os.listdir(traffic_path) if file.endswith("_traffic_data.tar")]

In [7]:
horas = [7,8,9,11,13,15,17,19,20,21]
traffic_data = dict([(h,[]) for h in horas])

In [8]:
for f in traffic_files:
    hora = int(f.split('_')[4].split('-')[0])
    if hora in horas:
        traffic_data[hora].append(f)

In [47]:
# Pondera las aristas según el tráfico, momentaneamente se asigna un valor de 0 a las aristas bloqueadas
weight_max = 2e20
map_vel = {1:1, 2: 2, 3: 4, 4: 6.7, 5: weight_max, 6: weight_max}
velocidades = [20,30,40,50,60]

k = []


for h in horas:

    linea = str(h) + ' & '
    
    mean_traffic = sum([pd.read_pickle(os.path.join(traffic_path, t)) for t in traffic_data[h]])/len(traffic_data[h])
    gdf_edges_h = copy(gdf_edges)
    gdf_nodes_h = copy(gdf_nodes)
    gdf_nodes_h['cell'] = 0

    gdf_edges_h = gdf_edges_h.assign(traffic = mean_traffic.round().astype(int)) 
    gdf_edges_h['weight'] = gdf_edges_h.apply(lambda x: x['length'] * map_vel[ x['traffic']], axis=1)

    # Se descartan las aristas con tráfico > 2
    pltbs_a_descartar = set()
    aristas_trafico_d = [(u,v,k) for u,v,k in gdf_edges_h[gdf_edges_h['traffic'] > 2].index.to_series()]
    for u,v,k in aristas_trafico_d:
        
        # Si el nodo v es una PLTB, se agrega a la lista de PLTBs a descartar
        if v in pltbs_nodos:
            pltbs_a_descartar.add(v)
        if u in pltbs_nodos:
            pltbs_a_descartar.add(u)
    
    linea += str(len(pltbs_a_descartar)) + ' '
    
    pltbs_h = list(set(pltbs_nodos) - pltbs_a_descartar)
    g_h = ox.graph_from_gdfs(gdf_nodes, gdf_edges_h)
    voronoi_h = voronoi_cells(g_h, pltbs_h, weight='weight', pred=True, length=True)
    
    dists = pd.Series(voronoi_h['length'])
    dists[dists == 0] = weight_max
    dists = dists[dists < weight_max]
    dists = dists.round(4)

    for vel in velocidades:
        m_x_min = vel*1000/60
        max_time = 9 #minutos


        dists_v = copy(dists)/m_x_min

        dists_v[ dists_v > max_time] = max_time + 1
        aceptable = round(sum(dists_v <= max_time)/len(dists_v),2)

        #dists.hist(bins = (max_time+1)*2)
        p95 = round(dists_v.quantile(q=0.95),2)

        linea += str(aceptable) + ' & ' + str(p95) + ' & '
    
    linea += '\n'
    print(linea)



7 & 47 0.96 & 8.15 & 0.99 & 5.44 & 0.99 & 4.08 & 1.0 & 3.26 & 1.0 & 2.72 & 

8 & 69 0.96 & 8.25 & 0.99 & 5.5 & 0.99 & 4.13 & 1.0 & 3.3 & 1.0 & 2.75 & 

9 & 66 0.96 & 8.17 & 0.99 & 5.45 & 1.0 & 4.09 & 1.0 & 3.27 & 1.0 & 2.72 & 

11 & 52 0.96 & 8.09 & 0.99 & 5.39 & 1.0 & 4.04 & 1.0 & 3.24 & 1.0 & 2.7 & 

13 & 74 0.96 & 8.39 & 0.99 & 5.59 & 1.0 & 4.19 & 1.0 & 3.36 & 1.0 & 2.8 & 



In [ ]:
pltbs_a_descartar = set()
aristas_trafico_d = [(u,v,k) for u,v,k in gdf_edges_h[gdf_edges_h['traffic'] > 2].index.to_series()]
# En el caso del tráfico, se descartan las aristas que no tienen tráfico libre 
for u,v,k in aristas_trafico_d:
    
    # Si el nodo v es una PLTB, se agrega a la lista de PLTBs a descartar
    if v in pltbs_nodos:
        pltbs_a_descartar.add(v)
    if u in pltbs_nodos:
        pltbs_a_descartar.add(u)
        
pltbs_nodos

In [41]:
pltbs_nodos_h = list(set(pltbs_nodos) - pltbs_a_descartar)

In [ ]:
gdf_nodes_h.plot(column='cell', cmap='RdPu',
                 figsize=(5,17/2),
                 markersize=2,)